In [19]:
import plotly as py
from plotly.grid_objs import Grid, Column

import time, os
import numpy as np
import matplotlib.image as img

from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)


WD = os.getcwd()
print(WD)

import glob
list_of_files = glob.glob('./data/PNG/SE000000/*.png')
list_of_files

pictureMRT=np.zeros((160,160,len(list_of_files)))
i=0
for file in list_of_files:
    pictureMRT[:,:,i] = img.imread(list_of_files[i])
    i = i+1
    
print ("Total of %d DICOM images\n" % pictureMRT.shape[2])
print ("Dimensons: %d x %d\n" % (pictureMRT.shape[0:2]))


/home/silvio/Dokumente/GitHub/BrainMRT
Total of 128 DICOM images

Dimensons: 160 x 160



In [29]:
deltaX = pictureMRT.shape[0] / pictureMRT.shape[2]
deltaY = pictureMRT.shape[1] / pictureMRT.shape[2]

x = np.linspace(0, pictureMRT.shape[0],deltaX)
y = np.linspace(0, pictureMRT.shape[1],deltaY)

r,c =pictureMRT.shape[0:2]

pictureMRT[1]

pl_bone = [
    [0.0, 'rgb(0, 0, 0)'],
    [0.05, 'rgb(10, 10, 14)'],
    [0.1, 'rgb(21, 21, 30)'],
    [0.15, 'rgb(33, 33, 46)'],
    [0.2, 'rgb(44, 44, 62)'],
    [0.25, 'rgb(56, 55, 77)'],
    [0.3, 'rgb(66, 66, 92)'],
    [0.35, 'rgb(77, 77, 108)'],
    [0.4, 'rgb(89, 92, 121)'],
    [0.45, 'rgb(100, 107, 132)'],
    [0.5, 'rgb(112, 123, 143)'],
    [0.55, 'rgb(122, 137, 154)'],
    [0.6, 'rgb(133, 153, 165)'],
    [0.65, 'rgb(145, 169, 177)'],
    [0.7, 'rgb(156, 184, 188)'],
    [0.75, 'rgb(168, 199, 199)'],
    [0.8, 'rgb(185, 210, 210)'],
    [0.85, 'rgb(203, 221, 221)'],
    [0.9, 'rgb(220, 233, 233)'],
    [0.95, 'rgb(238, 244, 244)'],
    [1.0, 'rgb(255, 255, 255)']
]

pictureMRT.shape[2]

128

In [30]:
my_columns = []
nr_frames = pictureMRT.shape[2]
for k in range(nr_frames):
    my_columns.extend(
        [Column((nr_frames - k * 0.1) * np.ones((r, c)), 'z{}'.format(k + 1)),
         Column(np.flipud(pictureMRT[nr_frames - k]), 'surfc{}'.format(k + 1))]
    )
    
    
grid = Grid(my_columns)
grid
#py.grid_ops.upload(grid, 'anim_sliceshead'+str(time.time()), auto_open=False)

[Column("[[ 128.  128.  128. ...,  128.  128.  128.]
 [ 128.  128.  128. ...,  128.  128.  128.]
 [ 128.  128.  128. ...,  128.  128.  128.]
 ..., 
 [ 128.  128.  128. ...,  128.  128.  128.]
 [ 128.  128.  128. ...,  128.  128.  128.]
 [ 128.  128.  128. ...,  128.  128.  128.]]", z1), Column("[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]]", surfc1), Column("[[ 127.9  127.9  127.9 ...,  127.9  127.9  127.9]
 [ 127.9  127.9  127.9 ...,  127.9  127.9  127.9]
 [ 127.9  127.9  127.9 ...,  127.9  127.9  127.9]
 ..., 
 [ 127.9  127.9  127.9 ...,  127.9  127.9  127.9]
 [ 127.9  127.9  127.9 ...,  127.9  127.9  127.9]
 [ 127.9  127.9  127.9 ...,  127.9  127.9  127.9]]", z2), Column("[[ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0.  0.]
 [ 0.  0.  0. ...,  0.  0. 

In [34]:
grid.get_column_reference('z1')

''

In [32]:
data=[
    dict(
        type='surface', 
        zsrc=grid.get_column_reference('z1'),
        surfacecolorsrc=grid.get_column_reference('surfc1'),
        colorscale=pl_bone,
        colorbar=dict(thickness=20, ticklen=4)
    )
]

frames=[]
for k in range(nr_frames):
    frames.append(
        dict(
            data=[dict(zsrc=grid.get_column_reference('z{}'.format(k + 1)),
                       surfacecolorsrc=grid.get_column_reference('surfc{}'.format(k + 1)))],
            name='frame{}'.format(k + 1)
        )
    )

sliders=[
    dict(
        steps=[dict(method='animate',
                    args= [['frame{}'.format(k + 1)],
                            dict(mode='immediate',
                                 frame= dict(duration=70, redraw= False),
                                 transition=dict(duration=0))],
                    label='{:d}'.format(k+1)) for k in range(68)], 
        transition= dict(duration=0),
        x=0,
        y=0, 
        currentvalue=dict(font=dict(size=12), 
                          prefix='slice: ', 
                          visible=True, 
                          xanchor='center'
                         ),  
        len=1.0
    )
]

axis3d = dict(
    showbackground=True, 
    backgroundcolor="rgb(230, 230,230)",
    gridcolor="rgb(255, 255, 255)",      
    zerolinecolor="rgb(255, 255, 255)",  
)

layout3d = dict(
         title='Slices in volumetric data', 
         font=dict(family='Balto'),
         width=600,
         height=600,
         scene=dict(xaxis=(axis3d),
                    yaxis=(axis3d), 
                    zaxis=dict(axis3d, 
                               **dict(range=[-0.1, 6.8], 
                                      autorange=False
                                     )
                              ), 
                    aspectratio=dict(x=1, 
                                     y=1, 
                                     z=1
                                    ),
                    ),
         updatemenus=[
             dict(type='buttons',
                  showactive=False,
                  y=1,
                  x=1.3,
                  xanchor='right',
                  yanchor='top',
                  pad=dict(t=0, r=10),
                  buttons=[dict(label='Play',
                                method='animate',
                                args=[
                                    None, 
                                    dict(frame=dict(duration=70, redraw=False),
                                         transition=dict(duration=0),
                                         fromcurrent=True,
                                         mode='immediate')
                                ])])
         ],
        sliders=sliders
)


In [33]:
fig=dict(data=data, layout=layout3d, frames=frames)
py.plotly.iplot(fig, 
                filename='sliceHead'+str(time.time()),
                validate=True)

ValueError: Invalid properties specified for object of type plotly.graph_objs.Scatter: ('zsrc', 'surfacecolorsrc')

    Valid properties:
        cliponaxis
            Determines whether or not markers and text nodes are
            clipped about the subplot axes. To show markers and
            text nodes above axis lines and tick labels, make sure
            to set `xaxis.layer` and `yaxis.layer` to *below
            traces*.
        connectgaps
            Determines whether or not gaps (i.e. {nan} or missing
            values) in the provided data arrays are connected.
        customdata
            Assigns extra data each datum. This may be useful when
            listening to hover, click and selection events. Note
            that, "scatter" traces also appends customdata items in
            the markers DOM elements
        customdatasrc
            Sets the source reference on plot.ly for  customdata .
        dx
            Sets the x coordinate step. See `x0` for more in